In [9]:
#!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

In [10]:
DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480

import math
import cv2 as cv
import matplotlib.pyplot as plt

def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  plt.imshow(img)

In [19]:
import mediapipe as mp

video = cv.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    while(True):
        ret, frame = video.read()
        if ret == True:
            frame = cv.flip(frame, 1)
            cv.imshow("Salida", frame)

            # Flip the image horizontally for a later selfie-view display
            image = cv.cvtColor(cv.flip(frame, 1), cv.COLOR_BGR2RGB)
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            # Process the image and find hand landmarks
            results = hands.process(image)

            # Draw the hand landmarks on the image
            image.flags.writeable = True
            image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:
              for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                print(hand_landmarks)
            cv.imshow('MediaPipe Hands', image)
        if ret == False:
            video.set(cv.CAP_PROP_POS_FRAMES, 0)
        if cv.waitKey(10) & 0xFF == 27: break

video.release()
cv.destroyAllWindows()

landmark {
  x: 0.20861296355724335
  y: 0.49611717462539673
  z: 1.289016324790282e-07
}
landmark {
  x: 0.1630527824163437
  y: 0.5124372243881226
  z: 0.007548418361693621
}
landmark {
  x: 0.12780028581619263
  y: 0.5145964026451111
  z: 0.005718591157346964
}
landmark {
  x: 0.10404251515865326
  y: 0.517780065536499
  z: 0.0010732596274465322
}
landmark {
  x: 0.08093559741973877
  y: 0.5210629105567932
  z: -0.0045396191999316216
}
landmark {
  x: 0.11668851971626282
  y: 0.4695446789264679
  z: -0.004651056136935949
}
landmark {
  x: 0.08269312977790833
  y: 0.4738035202026367
  z: -0.015605365857481956
}
landmark {
  x: 0.05741141736507416
  y: 0.48202186822891235
  z: -0.023957328870892525
}
landmark {
  x: 0.03579416871070862
  y: 0.4915691614151001
  z: -0.030305014923214912
}
landmark {
  x: 0.12282569706439972
  y: 0.46447253227233887
  z: -0.016012640669941902
}
landmark {
  x: 0.08429711312055588
  y: 0.46769842505455017
  z: -0.029107321053743362
}
landmark {
  x: 0.05